# Filtrado Colaborativo: *K-Nearest Neighbors*

Los algoritmos de **filtrado colaborativo** basados en la técnica de los *k* vecinos (KNN) tratan de imitar el comportamiento de los seres humanos cuando buscan recibir una recomendación: cuando necesitamos conocer si nos va a interesar un item, preguntamos a personas que sabemos que conocen nuestros intereses si ellos consideran que el item nos va a gustar.

El método realizará esta misma operación empleando la matriz de votaciones. Este proceso seguirá el siguiente algoritmo:

1. Determinar la similaridad entre los usuarios
2. Encontrar el conjunto de *k* usuarios más similares (*k* vecinos)
3. Estimar las predicciones a los items no votados utilizando las votaciones realizadas por los *k* vecinos
4. (Opcional) Recomendar los *N* items con una predicción más alta

En los siguientes subapartados explicaremos en detalle cada uno de estos pasos.

## Carga del dataset

Para ilustrar mejor el funcionamiento de la técnica de KNN, vamos a desarrollar una implementación explicativa de cómo funciona.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.

Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [1]:
import urllib.request
import math

In [ ]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

pene


Cargamos el dataset en la matriz de votaciones. La ausencia de voto se representa con None:

In [3]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

training_file = urllib.request.urlopen("https://drive.upm.es/s/tDdluElfGInyUnU/download")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

Del mismo modo, cargamos la matriz de votaciones de test:

In [4]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

test_file = urllib.request.urlopen("https://drive.upm.es/s/Jn75Vg6okOPsgZu/download")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

Definimos también algunas funciones auxiliares que nos serán útiles:

In [5]:
def rating_average (u):
  acc = 0
  count = 0

  for i in range(NUM_ITEMS):
    if ratings[u][i] != None:
      acc += ratings[u][i]
      count += 1

  avg = acc / count

  return avg

## Calculo de la similaridad

El primer paso del algoritmo de KNN consiste en determinar el parecido de cada pareja de usuarios basándonos en las votaciones previas de dichos usuarios. Para calcular esta similaridad debemos tener presente que la matriz de votaciones es dispersa y, por lo tanto, parar comparar dos usuarios únicamente podremos emplear los votos sobre aquellos items que sean comunes a ambos usuarios.

Por ejemplo, si un usuario *u* ha realizado las siguientes votaciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|  1 	|  2 	|    	|  4 	|  2 	|    	|  3 	|    	|   	|   5  	|

Y un usuario *v* ha realizado las siguientes votaciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| v 	|  3 	|   	|  2  	|   	|  5 	|  4  	|  3 	|  3  	|   	|  1 	|

Únicamente podremos comparar los votos producidos en los items *i1*, *i5*, *i7* e *i10*.

Existen infinidad de métricas de similaridad que permiten conocer el parecido de dos usuarios en función de sus votos comunes. Las más tradicionales se basan en medidas estadísticas clásicas. Por ejemplo, una de las métricas más empleadas es la **correlación**:

$$sim(u, v) = \frac {
    \sum_{i \in I_{u,v}} (r_{u,i} - \bar{r}_u)(r_{v,i} - \bar{r}_v)
   } {
   \sqrt{ \sum_{i \in I_{u,v}} (r_{u,i} - \bar{r}_u)^2 \sum_{i \in I_{u,v}} (r_{v,i} - \bar{r}_v)^2 }
   }$$
   
Donde $I_{u,v}$ representa los items que han votado tanto el usuario $u$ como el usuario $v$, $\bar{r}_u$ representa la votación media del usuario $u$ y $\bar{r}_v$ representa la votación media del usuario $v$.

La implementación de esta función podríamos hacerla del siguiente modo:

In [6]:
def correlation_similarity (u, v):
  num = 0

  den_u = 0
  den_v = 0

  count = 0

  avg_u = rating_average(u)
  avg_v = rating_average(v)

  for i in range(NUM_ITEMS):
    if ratings[u][i] != None and ratings[v][i] != None:
      r_u = ratings[u][i]
      r_v = ratings[v][i]

      num += (r_u - avg_u) * (r_v - avg_v)
      den_u += (r_u - avg_u) * (r_u - avg_u)
      den_v += (r_v - avg_v) * (r_v - avg_v)

      count += 1

  if count > 0 and den_u != 0 and den_v != 0:
    cor = num / math.sqrt( den_u * den_v )
    return cor;
  else:
    return None

In [8]:
# Testing
print(correlation_similarity(5,940))
print(correlation_similarity(113,534))
print(correlation_similarity(92,355))

0.010617834767124239
-0.08282165590325245
None


Aunque las métricas de similaridad basadas en medidas estadísticas clásicas ofrecen valores "aceptables" para el método de los *k* vecinos, existen otras métricas de similaridad específicas para el dominio del filtrado colaborativo. Una de las más populares es **JMSD** que permite calcular la similaridad atendiendo a dos factores: la información no numérica de los votos y la información numérica de los mismos. *JMSD* se define como el producto del índice de Jaccard por la diferencia cuadrática media:

$$JMSD(u,v) = Jaccard(u,v) * (1 - MSD(u, v))$$

El *índice de Jaccard* es una medida que permite comparar la similitud de dos conjuntos. En este caso se emplea para comprobar si los items votados por ambos usuarios coinciden, independientemente de la votación:

$$Jaccard(u,v) =\frac {I_u \cap I_v} {I_u \cup I_v} = \frac {\# \{ i \in I | r_{u,i} \neq \bullet \wedge r_{v,i} \neq \bullet \}} {\# \{ i \in I | r_{u,i} \neq \bullet \vee r_{v,i} \neq \bullet \}}$$

Donde $I_u$ representa los items votados por el usuario $u$ e $I_v$ representa los items votados por el usuario $v$.

El *MSD* permite comparar si los usuarios tienen la misma opinión sobre los items que votaron en común. En este caso, el *MSD* retorna el valor 0 cuando los usuarios votaron idéntico, por lo que suele invertirse para lograr que cuanto más se parezcan dos usuarios, mayor sea su similaridad. Para esto, es necesario normalizar previamente las votaciones:

$$MSD(u,v) = \frac {1} {\#I_{u,v}} \sum_{i \in I_{u,v}} (r_{u,i} - r_{v,i})^2$$

Donde $I_{u,v}$ representa los items votados en común por $u$ y $v$.

Esta medida busca un equilibrio entre los items que los dos usuarios han votado y lo parecidas que son estas votaciones.

La implementación de esta función podríamos hacerla del siguiente modo:

In [ ]:
def jmsd_similarity (u, v):

  union = 0
  intersection = 0
  diff = 0

  for i in range(NUM_ITEMS):
    if ratings[u][i] != None and ratings[v][i] != None:
      r_u = (ratings[u][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)
      r_v = (ratings[v][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)

      diff = (r_u - r_v) * (r_u - r_v)

      intersection += 1
      union += 1

    elif ratings[u][i] != None or ratings[v][i] != None:
      union += 1


  if intersection > 0:
    jaccard = intersection / union
    msd = diff / intersection
    return jaccard * (1 - msd);
  else:
    return None

In [ ]:
# Testing
print(jmsd_similarity(5,940))
print(jmsd_similarity(113,534))
print(jmsd_similarity(92,355))

0.049268018018018014
0.12685381355932204
None


## Búsqueda de los *k* vecinos

En el método de KNN, las predicciones se realizan a partir del conjunto de *k* vecinos de cada usuario, esto es, el conjunto de *k* usuarios más similares a uno dado. Este *k* será considerado como un parámetro del sistema y deberá tunearse para cada dataset.

La búsqueda de los *k* vecinos consiste únicamente en ordenar los usuarios en base a su similaridad y elegir a los *k* con una similaridad más alta.

La implementación de esta función podríamos hacerla del siguiente modo:

In [ ]:
k = 25

In [ ]:
def get_neighbors (u, similarities):

  neighbors = [None for _ in range(k)]

  for n in range(k):

    max_similarity = 0
    neighbor = None

    for v, sim in enumerate(similarities):
      if v not in neighbors and sim != None and sim > max_similarity:
        max_similarity = sim
        neighbor = v

    neighbors[n] = neighbor

  return neighbors

In [ ]:
# Testing
u = 112
similarities_corr = [None if u == v else correlation_similarity(u, v) for v in range(NUM_USERS)]
similarities_jmsd = [None if u == v else jmsd_similarity(u, v) for v in range(NUM_USERS)]
neighbors_corr = get_neighbors(u, similarities_corr)
neighbors_jmsd = get_neighbors(u, similarities_jmsd)
print(neighbors_corr)
print(neighbors_jmsd)

[97, 50, 66, 113, 121, 186, 318, 365, 515, 521, 699, 40, 507, 659, 606, 87, 703, 797, 856, 134, 722, 875, 132, 379, 608]
[936, 172, 769, 169, 742, 175, 73, 577, 469, 412, 581, 2, 148, 702, 62, 489, 154, 790, 833, 509, 729, 46, 925, 700, 643]


## Estimación de las predicciones

La estimación de las predicciones se realiza agregando las votaciones que los *k* vecinos del usuario activo realizaron al item que se quiere predecir. De nuevo, nos encontramos con el problema de la dispersión de la matriz de votaciones, ya que no todos los *k* vecinos habrán votado los mismos items. Podría incluso darse la circunstancia de que un item no hubiera sido votado por ninguno de los *k* vecinos y, por tanto, no podría estimarse una predicción.

Esta situación puede ayudarnos a tunear el parámetro *k*. Si elegimos un *k* muy bajo conseguimos predicciones precisas ya que se realizan con usuarios muy similares al activo, pero existirán muchos items que no podremos predecir. Si elegimos un *k* muy elevado podremos predecir casi todos los items pero estas predicciones serán poco personalizadas. Llevado al extremo si *k* es igual al número de usuario del sistema (menos uno), las predicciones serían equivalentes a la media de los votos.

Existen diversas formas de agregar las votaciones de los *k* vecinos (*aggregation approach*), aunque, la más popular debido a su sencillez es la media:

$$\hat{r}_{u,i} = \frac{1}{\#N_{u,i}} \sum_{n \in N_{u,i}} r_{n,i}$$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$.

La implementación de esta medida de agregación podríamos hacerla del siguiente modo:

In [ ]:
def average_prediction (u, i, neighbors):
  acc = 0
  count = 0

  for n in neighbors:
    if n == None: break

    if ratings[n][i] != None:
      acc += ratings[n][i]
      count += 1

  if count > 0:
    prediction = acc / count
    return prediction
  else:
    return None


In [ ]:
# Testing
i = 324
avgpred_corr = average_prediction (u, i, neighbors_corr)
avgpred_jmsd = average_prediction (u, i, neighbors_jmsd)
print(avgpred_corr)
print(avgpred_jmsd)

1.0
1.75


Una evolución simple de esta medida de agregación es la media ponderada, en la cual el voto de cada uno de los vecinos se pondera en función de su similaridad con el usuario sobre el que se está calculando la predicción:

$$\hat{r}_{u,i} = \frac{\sum_{n \in N_{u,i}} sim(u,n) \cdot r_{n,i}}{\sum_{n \in N_{u,i}} sim(u,n)} $$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$ y $sim(u,n)$ simboliza la similaridad entre el usuario $u$ y el vecino $n$.

In [ ]:
def weighted_average_prediction (u, i, neighbors, similarities):
  num = 0
  den = 0

  for n in neighbors:
    if n == None: break

    if ratings[n][i] != None:
      num += similarities[n] * ratings[n][i]
      den += similarities[n]

  if den > 0:
    prediction = num / den
    return prediction
  else:
    return None

In [ ]:
# Testing
wavgpred_corr = weighted_average_prediction (u, i, neighbors_corr, similarities_corr)
wavgpred_jmsd = weighted_average_prediction (u, i, neighbors_jmsd, similarities_jmsd)
print(wavgpred_corr)
print(wavgpred_jmsd)

1.0
1.786271948118128


Utilizar la media o la media ponderada como medida de agregación tiene un problema, presupone que todos los usuarios tienen la misma percepción de la escala de votaciones prefijada. Sin embargo, sabemos que esto no es cierto. Existes determinados sesgos que hacen que los usuarios realicen votaciones haciendo una interpretación particular del sentido de su voto. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.

Para incluir este fenómeno dentro de las medidas de agregación, es frecuente agregar las votaciones de los k vecinos mediante la **desviación respecto a la media** (*deviation from mean*):

$$\hat{r}_{u,i} = \bar{r}_{u} + \frac{ \sum_{n \in N_{u,i}} r_{n,i} - \bar{r}_n }{\#N_{u,i}}$$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$, $\bar{r}_u$ representa la media de votos del usuario $u$ y $\bar{r}_n$ representa la media de votos del usuario $n$.

La implementación de esta medida de agregación podríamos hacerla del siguiente modo:

In [ ]:
def deviation_from_mean_prediction (u, i, neighbors):
  acc = 0
  count = 0

  for n in neighbors:
    if n == None: break

    if ratings[n][i] != None:
      avg_n = rating_average(n)
      acc += ratings[n][i] - avg_n
      count += 1

  if count > 0:
    avg_u = rating_average(u)
    prediction = avg_u + acc / count
    return prediction
  else:
    return None

In [ ]:
# Testing
dfmpred_corr = deviation_from_mean_prediction (u, i, neighbors_corr)
dfmpred_jmsd = deviation_from_mean_prediction (u, i, neighbors_jmsd)
print(dfmpred_corr)
print(dfmpred_jmsd)

1.4452214452214451
2.367587967587968


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

En algunas ocasiones, es posible establecer filtros para acotar los items a recomendar. Por ejemplo: en un sistema de recomendación de restaurantes, es posible filtrar aquellos items que se encuentren a demasiada distancia del usuarios; en un sistema de recomendación de libros, el usuario puede filtrar el idioma o el género literario del libro; en una web de comercio electrónico es posible realizar recomendaciones sobre una categoría concreta...


In [ ]:
N = 5

In [ ]:
def get_recommendations (predictions):
  recommendations = [None for _ in range(N)]

  for n in range(N):

    max_value = 0
    item = None

    for i, value in enumerate(predictions):
      if i not in recommendations and value != None and value > max_value:
        max_value = value
        item = i

    recommendations[n] = item

  return recommendations

In [ ]:
# Testing
avg_predictions_corr = [None if ratings[u][i] != None else average_prediction(u, i, neighbors_corr) for i in range(NUM_ITEMS)]
avg_predictions_jmsd = [None if ratings[u][i] != None else average_prediction(u, i, neighbors_jmsd) for i in range(NUM_ITEMS)]
wavg_predictions_corr = [None if ratings[u][i] != None else weighted_average_prediction(u, i, neighbors_corr, similarities_corr) for i in range(NUM_ITEMS)]
wavg_predictions_jmsd = [None if ratings[u][i] != None else weighted_average_prediction(u, i, neighbors_jmsd, similarities_jmsd) for i in range(NUM_ITEMS)]
dfm_predictions_corr = [None if ratings[u][i] != None else deviation_from_mean_prediction(u, i, neighbors_corr) for i in range(NUM_ITEMS)]
dfm_predictions_jmsd = [None if ratings[u][i] != None else deviation_from_mean_prediction(u, i, neighbors_jmsd) for i in range(NUM_ITEMS)]
recommendations_avg_corr = get_recommendations(avg_predictions_corr)
recommendations_avg_jmsd = get_recommendations(avg_predictions_jmsd)
recommendations_wavg_corr = get_recommendations(wavg_predictions_corr)
recommendations_wavg_jmsd = get_recommendations(wavg_predictions_jmsd)
recommendations_dfm_corr = get_recommendations(dfm_predictions_corr)
recommendations_dfm_jmsd = get_recommendations(dfm_predictions_jmsd)
print(recommendations_avg_corr)
print(recommendations_avg_jmsd)
print(recommendations_wavg_corr)
print(recommendations_wavg_jmsd)
print(recommendations_dfm_corr)
print(recommendations_dfm_jmsd)

##Ejemplo de ejecución: cálculo del MAE

En esta sección vamos a mostrar el ejemplo completo de cómo calcular el error medio absoluto (MAE) de las predicciones realizadas por el método de los *k*-vecinos.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [ ]:
def has_test_ratings (u):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      return True
  return False

In [ ]:
avg_predictions_corr = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
avg_predictions_jmsd = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
wavg_predictions_corr = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
wavg_predictions_jmsd = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
dfm_predictions_corr = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
dfm_predictions_jmsd = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
  if has_test_ratings(u):

    # Calcular similaridades
    similarities_corr = [None if u == v else correlation_similarity(u, v) for v in range(NUM_USERS)]
    similarities_jmsd = [None if u == v else jmsd_similarity(u, v) for v in range(NUM_USERS)]

    # Calcular vecinos
    neighbors_corr = get_neighbors(u, similarities_corr)
    neighbors_jmsd = get_neighbors(u, similarities_jmsd)

    # Calcular predicciones sobre los items de test votados por el usuario
    for i in range(NUM_ITEMS):
      if test_ratings[u][i] != None:
        avg_predictions_corr[u][i] = average_prediction(u, i, neighbors_corr)
        avg_predictions_jmsd[u][i] = average_prediction(u, i, neighbors_jmsd)
        wavg_predictions_corr[u][i] = weighted_average_prediction(u, i, neighbors_corr, similarities_corr)
        wavg_predictions_jmsd[u][i] = weighted_average_prediction(u, i, neighbors_jmsd, similarities_jmsd)
        dfm_predictions_corr[u][i] = deviation_from_mean_prediction(u, i, neighbors_corr)
        dfm_predictions_jmsd[u][i] = deviation_from_mean_prediction(u, i, neighbors_jmsd)

Y, a continuación, calculamos el MAE:

In [ ]:
def get_user_mae (u, predictions):
  mae = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return mae / count
  else:
    return None

In [ ]:
def get_mae (predictions):
  mae = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_mae = get_user_mae(u, predictions)

      if user_mae != None:
        mae += user_mae
        count += 1


  if count > 0:
    return mae / count
  else:
    return None

In [ ]:
mae_avg_corr = get_mae(avg_predictions_corr)
mae_avg_jmsd = get_mae(avg_predictions_jmsd)
mae_wavg_corr = get_mae(wavg_predictions_corr)
mae_wavg_jmsd = get_mae(wavg_predictions_jmsd)
mae_dfm_corr = get_mae(dfm_predictions_corr)
mae_dfm_jmsd = get_mae(dfm_predictions_jmsd)
print("MAE Avg Prediction with Corr Similarity = " + str(mae_avg_corr))
print("MAE Avg Prediction with JMSD Similarity = " + str(mae_avg_jmsd))
print("MAE Weighted Avg Prediction with Corr Similarity = " + str(mae_wavg_corr))
print("MAE Weighted Avg Prediction with JMSD Similarity = " + str(mae_wavg_jmsd))
print("MAE Dev from Mean Prediction with Corr Similarity = " + str(mae_dfm_corr))
print("MAE Dev from Mean Prediction with JMSD Similarity = " + str(mae_dfm_jmsd))

MAE Avg Prediction with Corr Similarity = 1.00923070340394
MAE Avg Prediction with JMSD Similarity = 0.8951441319983335
MAE Weighted Avg Prediction with Corr Similarity = 1.0097877459011548
MAE Weighted Avg Prediction with JMSD Similarity = 0.8955826103175067
MAE Dev from Mean Prediction with Corr Similarity = 1.008429618748919
MAE Dev from Mean Prediction with JMSD Similarity = 0.828759929568012


# Cálculo del resto de medidas

Calcula el resto de medidas (RMSE, Precision, Recall, F1 y nDCG) y compara las distintas combinaciones de funciones de similitud y de agregación para saber cuál combinación proporciona mejores recomendaciones.

In [ ]:
def get_user_rmse (u, predictions):
  mse = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mse += (test_ratings[u][i] - predictions[u][i]) * (test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return math.sqrt(mse / count)
  else:
    return None

In [ ]:
def get_rmse (predictions):
  rmse = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_rmse = get_user_rmse(u, predictions)

      if user_rmse != None:
        rmse += user_rmse
        count += 1


  if count > 0:
    return rmse / count
  else:
    return None

In [ ]:
rmse_avg_corr = get_rmse(avg_predictions_corr)
rmse_avg_jmsd = get_rmse(avg_predictions_jmsd)
rmse_wavg_corr = get_rmse(wavg_predictions_corr)
rmse_wavg_jmsd = get_rmse(wavg_predictions_jmsd)
rmse_dfm_corr = get_rmse(dfm_predictions_corr)
rmse_dfm_jmsd = get_rmse(dfm_predictions_jmsd)
print("RMSE Avg Prediction with Corr Similarity = " + str(rmse_avg_corr))
print("RMSE Avg Prediction with JMSD Similarity = " + str(rmse_avg_jmsd))
print("RMSE Weighted Avg Prediction with Corr Similarity = " + str(rmse_wavg_corr))
print("RMSE Weighted Avg Prediction with JMSD Similarity = " + str(rmse_wavg_jmsd))
print("RMSE Dev from Mean Prediction with Corr Similarity = " + str(rmse_dfm_corr))
print("RMSE Dev from Mean Prediction with JMSD Similarity = " + str(rmse_dfm_jmsd))

RMSE Avg Prediction with Corr Similarity = 1.2145699492647715
RMSE Avg Prediction with JMSD Similarity = 1.0887466613587198
RMSE Weighted Avg Prediction with Corr Similarity = 1.215036240076365
RMSE Weighted Avg Prediction with JMSD Similarity = 1.0878343827807315
RMSE Dev from Mean Prediction with Corr Similarity = 1.1827392506731436
RMSE Dev from Mean Prediction with JMSD Similarity = 1.0208335949966347


In [ ]:
theta = 4

In [ ]:
def get_user_precision (u, predictions):
  precision = 0
  count = 0
  recommendations = get_recommendations(predictions[u])

  for i in recommendations:
    if i != None and test_ratings[u][i] != None:
      precision += 1 if test_ratings[u][i] >= theta else 0
      count += 1

  if count > 0:
    return precision / count
  else:
    return None

In [ ]:
def get_precision (predictions):
  precision = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_precision = get_user_precision(u, predictions)

      if user_precision != None:
        precision += user_precision
        count += 1


  if count > 0:
    return precision / count
  else:
    return None

In [ ]:
precision_avg_corr = get_precision(avg_predictions_corr)
precision_avg_jmsd = get_precision(avg_predictions_jmsd)
precision_wavg_corr = get_precision(wavg_predictions_corr)
precision_wavg_jmsd = get_precision(wavg_predictions_jmsd)
precision_dfm_corr = get_precision(dfm_predictions_corr)
precision_dfm_jmsd = get_precision(dfm_predictions_jmsd)
print("Precision Avg Prediction with Corr Similarity = " + str(precision_avg_corr))
print("Precision Avg Prediction with JMSD Similarity = " + str(precision_avg_jmsd))
print("Precision Weighted Avg Prediction with Corr Similarity = " + str(precision_wavg_corr))
print("Precision Weighted Avg Prediction with JMSD Similarity = " + str(precision_wavg_jmsd))
print("Precision Dev from Mean Prediction with Corr Similarity = " + str(precision_dfm_corr))
print("Precision Dev from Mean Prediction with JMSD Similarity = " + str(precision_dfm_jmsd))

Precision Avg Prediction with Corr Similarity = 0.6281578947368416
Precision Avg Prediction with JMSD Similarity = 0.6876923076923074
Precision Weighted Avg Prediction with Corr Similarity = 0.6249999999999994
Precision Weighted Avg Prediction with JMSD Similarity = 0.6897435897435894
Precision Dev from Mean Prediction with Corr Similarity = 0.6239473684210521
Precision Dev from Mean Prediction with JMSD Similarity = 0.6641025641025636


In [ ]:
def get_user_recall (u, predictions):
  recall = 0
  count = 0
  recommendations = get_recommendations(predictions[u])

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      if test_ratings[u][i] >= theta:
        recall += 1 if i in recommendations else 0
        count += 1

  if count > 0:
    return recall / count
  else:
    return None

In [ ]:
def get_recall (predictions):
  recall = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_recall = get_user_recall(u, predictions)

      if user_recall != None:
        recall += user_recall
        count += 1


  if count > 0:
    return recall / count
  else:
    return None

In [ ]:
recall_avg_corr = get_recall(avg_predictions_corr)
recall_avg_jmsd = get_recall(avg_predictions_jmsd)
recall_wavg_corr = get_recall(wavg_predictions_corr)
recall_wavg_jmsd = get_recall(wavg_predictions_jmsd)
recall_dfm_corr = get_recall(dfm_predictions_corr)
recall_dfm_jmsd = get_recall(dfm_predictions_jmsd)
print("Recall Avg Prediction with Corr Similarity = " + str(recall_avg_corr))
print("Recall Avg Prediction with JMSD Similarity = " + str(recall_avg_jmsd))
print("Recall Weighted Avg Prediction with Corr Similarity = " + str(recall_wavg_corr))
print("Recall Weighted Avg Prediction with JMSD Similarity = " + str(recall_wavg_jmsd))
print("Recall Dev from Mean Prediction with Corr Similarity = " + str(recall_dfm_corr))
print("Recall Dev from Mean Prediction with JMSD Similarity = " + str(recall_dfm_jmsd))

Recall Avg Prediction with Corr Similarity = 0.8688305006142428
Recall Avg Prediction with JMSD Similarity = 0.5559909977686648
Recall Weighted Avg Prediction with Corr Similarity = 0.8668824486661909
Recall Weighted Avg Prediction with JMSD Similarity = 0.5514566108031268
Recall Dev from Mean Prediction with Corr Similarity = 0.8657731598160424
Recall Dev from Mean Prediction with JMSD Similarity = 0.5397417491845959


In [ ]:
def get_user_f1 (u, predictions):
  precision = get_user_precision(u, predictions)
  recall = get_user_recall(u, predictions)

  if precision == None or recall == None:
    return None
  elif precision == 0 and recall == 0:
    return 0
  else:
    return 2 * precision * recall / (precision + recall)

In [ ]:
def get_f1 (predictions):
  f1 = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_f1 = get_user_f1(u, predictions)

      if user_f1 != None:
        f1 += user_f1
        count += 1


  if count > 0:
    return f1 / count
  else:
    return None

In [ ]:
f1_avg_corr = get_f1(avg_predictions_corr)
f1_avg_jmsd = get_f1(avg_predictions_jmsd)
f1_wavg_corr = get_f1(wavg_predictions_corr)
f1_wavg_jmsd = get_f1(wavg_predictions_jmsd)
f1_dfm_corr = get_f1(dfm_predictions_corr)
f1_dfm_jmsd = get_f1(dfm_predictions_jmsd)
print("F1 Avg Prediction with Corr Similarity = " + str(f1_avg_corr))
print("F1 Avg Prediction with JMSD Similarity = " + str(f1_avg_jmsd))
print("F1 Weighted Avg Prediction with Corr Similarity = " + str(f1_wavg_corr))
print("F1 Weighted Avg Prediction with JMSD Similarity = " + str(f1_wavg_jmsd))
print("F1 Dev from Mean Prediction with Corr Similarity = " + str(f1_dfm_corr))
print("F1 Dev from Mean Prediction with JMSD Similarity = " + str(f1_dfm_jmsd))

F1 Avg Prediction with Corr Similarity = 0.7218337038142232
F1 Avg Prediction with JMSD Similarity = 0.5342544051079944
F1 Weighted Avg Prediction with Corr Similarity = 0.7192647449952644
F1 Weighted Avg Prediction with JMSD Similarity = 0.5324311280164647
F1 Dev from Mean Prediction with Corr Similarity = 0.7183381057975211
F1 Dev from Mean Prediction with JMSD Similarity = 0.5174990149479768


In [ ]:
def get_ordered_test_items(u):
  num_items = sum(x is not None for x in test_ratings[u])
  items = [None for _ in range(num_items)]

  for n in range(num_items):

    max_value = 0
    item = None

    for i,value in enumerate(test_ratings[u]):
      if i not in items and value != None and value > max_value:
        max_value = value
        item = i

    items[n] = item

  return items

In [ ]:
def get_user_idcg (u):
  items = get_ordered_test_items(u)
  idcg = 0

  for pos, i in enumerate(items):
    idcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return idcg

In [ ]:
def get_user_dcg (u, recommendations):
  dcg = 0

  for pos, i in enumerate(recommendations):
    if i != None and test_ratings[u][i] != None:
      dcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return dcg

In [ ]:
def get_user_ndcg (u, predictions):
  recommendations = get_recommendations(predictions[u])
  dcg = get_user_dcg(u, recommendations)
  idcg = get_user_idcg(u)
  if idcg == 0:
    return 0
  else:
    return dcg / idcg

In [ ]:
def get_ndcg (predictions):
  ndcg = 0
  count = 0

  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_ndcg = get_user_ndcg(u, predictions)

      if user_ndcg != None:
        ndcg += user_ndcg
        count += 1


  if count > 0:
    return ndcg / count
  else:
    return None

In [ ]:
ndcg_avg_corr = get_ndcg(avg_predictions_corr)
ndcg_avg_jmsd = get_ndcg(avg_predictions_jmsd)
ndcg_wavg_corr = get_ndcg(wavg_predictions_corr)
ndcg_wavg_jmsd = get_ndcg(wavg_predictions_jmsd)
ndcg_dfm_corr = get_ndcg(dfm_predictions_corr)
ndcg_dfm_jmsd = get_ndcg(dfm_predictions_jmsd)
print("nDCG Avg Prediction with Corr Similarity = " + str(ndcg_avg_corr))
print("nDCG Avg Prediction with JMSD Similarity = " + str(ndcg_avg_jmsd))
print("nDCG Weighted Avg Prediction with Corr Similarity = " + str(ndcg_wavg_corr))
print("nDCG Weighted Avg Prediction with JMSD Similarity = " + str(ndcg_wavg_jmsd))
print("nDCG Dev from Mean Prediction with Corr Similarity = " + str(ndcg_dfm_corr))
print("nDCG Dev from Mean Prediction with JMSD Similarity = " + str(ndcg_dfm_jmsd))

nDCG Avg Prediction with Corr Similarity = 0.4159898617717814
nDCG Avg Prediction with JMSD Similarity = 0.5458288843769105
nDCG Weighted Avg Prediction with Corr Similarity = 0.4149921091918133
nDCG Weighted Avg Prediction with JMSD Similarity = 0.5447775527663378
nDCG Dev from Mean Prediction with Corr Similarity = 0.4137352449188077
nDCG Dev from Mean Prediction with JMSD Similarity = 0.5336847977402612


## Referencias

Bobadilla, J., Serradilla, F., & Bernal, J. (2010). **A new collaborative filtering metric that improves the behavior of recommender systems**. Knowledge-Based Systems, 23(6), 520-528.